In [1]:
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf

2023-04-09 11:27:00.067476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 11:27:00.987228: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# create a custom layer witch have a buffer to store the weights and informationb throw the training

class DenseBuffer(layers.Layer):

    def __init__(self, units=32, input_dim=32):
        super(DenseBuffer, self).__init__()
        self.buffer = None #the buffer is the res of the layer
        self.units = units
        self.input_dim = input_dim
        self.buffer_iterator = 0 #the iterator to iterate over the buffer
        #create an iteraor to iterate over the buffer


    def build(self, input_shape):

        w_init = tf.random_normal_initializer()

        self.w = tf.Variable(
            initial_value=w_init(shape=(input_shape[-1], self.units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(self.units,), dtype="float32"), trainable=True
        )

        #create the buffer
        self.buffer = tf.Variable(
            initial_value=tf.zeros(shape=(1, self.units), dtype="float32"),
            trainable=False,
        )

    def call(self, inputs):
        if self.buffer_iterator == 0:
            #if the buffer is empty, just add the res of the layer
            res = tf.matmul(inputs, self.w) + self.b
            self.buffer = res
            self.buffer_iterator += 1
            return res
        else:
            #if the buffer is not empty, add the res of the layer to the buffer
            res = tf.matmul(tf.matmul(inputs, self.w) + self.b, self.buffer)
            self.buffer = res
            self.buffer_iterator += 1
            return res

In [3]:
test = DenseBuffer()

In [5]:
test(tf.constant([[1, 2]], dtype=tf.float32))

2023-04-09 11:27:02.245695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-09 11:27:02.274272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-09 11:27:02.274324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-09 11:27:02.276709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-09 11:27:02.276760: I tensorflow/compile

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.00844032, -0.00092541, -0.00511014, -0.16888063, -0.08427596,
        -0.21597949, -0.01732077,  0.01453283, -0.02193583, -0.10942151,
         0.1308423 , -0.06047181, -0.2800917 , -0.00221445,  0.0289262 ,
        -0.1397514 ,  0.09553465,  0.05442366,  0.0678086 ,  0.13387525,
        -0.01041169,  0.0281101 ,  0.06670436, -0.09726016, -0.14215322,
        -0.20824023, -0.02130379,  0.09637992, -0.11698732, -0.08628592,
        -0.00962289, -0.11411408]], dtype=float32)>